In [13]:
import numpy as np; import pandas as pd; import pyodbc; import datetime; import sklearn; import os; import math
from sklearn import feature_selection, linear_model, grid_search, cross_validation, metrics, ensemble, learning_curve 
from sklearn.preprocessing import scale
from sklearn.manifold import TSNE
from datetime import datetime
from sklearn.cluster import KMeans
import pickle
import seaborn as sns
import xgboost as xgb
from sklearn.feature_selection import chi2, f_classif, SelectKBest
from sklearn.preprocessing import StandardScaler
pd.options.display.max_columns = 999
%pylab inline 
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")
    
from IPython.core.display import display,HTML
display(HTML("<style>.container{width:90% !important;}</style>"))

conn = pyodbc.connect(r'DRIVER={SQL Server Native Client 11.0};'r'SERVER=WIN-5G5AUOCEJPK;'r'DATABASE=Gold585;'r'Trusted_Connection=yes;')

Populating the interactive namespace from numpy and matplotlib


In [14]:
#Отбираем ПОСЛЕДНИЕ залоговые билеты со статусом Новый залог за последний год
sql_query = "SELECT  \
      a.[PartnerID] \
      ,[Age] \
      ,[SEX] \
      ,[CityID] \
      ,IIF(QntPurchaseCode=0,0,LifeTimeM*30/QntPurchaseCode) AvgDneyPrihodov \
      ,[LifeTimeM] + DATEDIFF(Month, PurchaseDateTime, GetDate()) [LifeTimeM] \
      ,[QntPurchaseCode] \
      ,[SumQuantity] \
      ,[SUMisPartnerPromotionID] \
      ,[PercentisPartnerPromotionID] \
      ,[AVGisPartnerPromotionID] \
      ,[SUMisCrossPromo] \
      ,[PercentisCrossPromo] \
      ,[AVGisCrossPromo] \
      ,[AVGPercentinSMS] \
      ,[PercentMonth1] \
      ,[PercentMonth2] \
      ,[PercentMonth3] \
      ,[PercentMonth4] \
      ,[PercentMonth5] \
      ,[PercentMonth6] \
      ,[PercentMonth7] \
      ,[PercentMonth8] \
      ,[PercentMonth9] \
      ,[PercentMonth10] \
      ,[PercentMonth11] \
      ,[PercentMonth12] \
      ,[PercentNY] \
      ,[PercentDR] \
      ,[PercentVD] \
      ,[Percent8M] \
      ,[SUMPrice] \
      ,[AVGPrice] \
      ,[SumSumma] \
      ,[AVGSumma] \
      ,[SumSummaDiscount] \
      ,[AvgSummaDiscount] \
      ,[AVGProcentSkidki] \
      ,[PercentMen] \
      ,[SumChild] \
      ,[PercentChild] \
      ,[PercentKolco] \
      ,[PercentSERGI] \
      ,[PercentPODVES] \
      ,[PercentFUTLAR] \
      ,[PercentCEP] \
      ,[PercentBRASLET] \
      ,[PercentKOLE] \
      ,[PercentNKNOWN_WEAR] \
      ,[AVGCostPrice] \
      ,[AVGCostMin] \
      ,[AVGCostTag] \
      ,[AVGCostStock] \
      ,[AVGCostRetail] \
      ,[AVGGiftCardPositionID] \
      ,[SumGiftCardPositionID] \
      ,[AVGQntName] \
      ,[PercentExpensive] \
      ,[PercentCheap] \
      ,[PercentPremium] \
      ,[PercentMidle] \
      ,[PercentGold] \
      ,[PercentSilver] \
      ,[PercentNoMetal] \
      ,b.labels \
  FROM [Gold585].[dbo].[RetailCommon] a \
  left join ClastersRetail b on a.PartnerID = b.PartnerID \
  WHERE a.partnerID <> 0 and PurchaseDateTime = (Select MAX(PurchaseDateTime) MaxDate From RetailCommon rt where rt.PartnerID = a.PartnerID) \
  order BY a.PartnerID, PurchaseDateTime"
data = pd.read_sql(sql_query,conn)
data.head()

,PartnerID,Age,SEX,CityID,AvgDneyPrihodov,LifeTimeM,QntPurchaseCode,SumQuantity,SUMisPartnerPromotionID,PercentisPartnerPromotionID,AVGisPartnerPromotionID,SUMisCrossPromo,PercentisCrossPromo,AVGisCrossPromo,AVGPercentinSMS,PercentMonth1,PercentMonth2,PercentMonth3,PercentMonth4,PercentMonth5,PercentMonth6,PercentMonth7,PercentMonth8,PercentMonth9,PercentMonth10,PercentMonth11,PercentMonth12,PercentNY,PercentDR,PercentVD,Percent8M,SUMPrice,AVGPrice,SumSumma,AVGSumma,SumSummaDiscount,AvgSummaDiscount,AVGProcentSkidki,PercentMen,SumChild,PercentChild,PercentKolco,PercentSERGI,PercentPODVES,PercentFUTLAR,PercentCEP,PercentBRASLET,PercentKOLE,PercentNKNOWN_WEAR,AVGCostPrice,AVGCostMin,AVGCostTag,AVGCostStock,AVGCostRetail,AVGGiftCardPositionID,SumGiftCardPositionID,AVGQntName,PercentExpensive,PercentCheap,PercentPremium,PercentMidle,PercentGold,PercentSilver,PercentNoMetal,labels
0,2,46,0,35,0,34,1,2,0,0,0,0,0.0,0,,1.0,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0.00,0.0,1.0,0.000,0.0,4320,4320.000,4320.0,4320.000,3540.0,3540.0,0.5496,0.0,0,0.0,0.000000,0.000000,1.0,0.0,0.000000,0.000000,0.0,0.0,1941.9200,2130.6000,7860.000,0.0,5400.000,0,0,1,0.00,0.0,0.0,1.00,1.0,0.000000,0.0,1.4
1,4,118,0,92,75,23,8,12,0,0,0,0,0.0,0,,0.0,0.125,0.0,0.0,0.0,0.125,0.125,0.375,0.0,0.0,0.0,0.25,0.0,0.0,0.125,0.0,8125,1015.625,8125.0,1015.625,5444.0,680.5,0.5937,0.0,0,0.0,0.166667,0.083333,0.5,0.0,0.083333,0.083333,0.0,0.0,442.8612,668.8425,1969.875,803.0,1071.125,0,0,1,0.25,0.5,0.0,0.25,0.0,0.916667,0.0,2.3
2,6,30,0,32,0,34,1,2,0,0,0,0,0.0,0,,1.0,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0.00,0.0,0.0,0.000,0.0,8952,8952.000,8952.0,8952.000,6168.0,6168.0,0.5920,0.0,0,0.0,1.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,4385.8300,5160.4500,15120.000,0.0,11190.000,0,0,2,0.00,0.0,0.0,1.00,1.0,0.000000,0.0,1.2
3,9,116,0,122,0,34,1,1,0,0,0,0,0.0,0,,1.0,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0.00,0.0,0.0,0.000,0.0,21310,21310.000,21310.0,21310.000,21450.0,21450.0,0.4983,0.0,0,0.0,1.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,11395.0000,3330.0000,42760.000,0.0,24310.000,0,0,1,1.00,0.0,0.0,0.00,1.0,0.000000,0.0,1.2
4,10,20,0,105,0,34,1,2,0,0,0,1,1.0,1,,1.0,0.000,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0.00,0.0,0.0,0.000,0.0,571,571.000,571.2,571.200,392.8,392.8,0.5925,0.0,0,0.0,0.500000,0.500000,0.0,0.0,0.000000,0.000000,0.0,0.0,194.1000,307.0000,1039.000,229.0,714.000,0,0,2,0.00,1.0,0.0,0.00,0.0,0.500000,0.5,1.3


In [35]:
data.describe(include='object').T

,count,unique,top,freq
AVGPercentinSMS,2778321,674,,2553156
labels,2521444,12,1.2,612507


In [36]:
data['labels'].nc

1.2    612507
1.3    473692
2.2    373053
1.1    302595
2.3    254818
1.4    175388
2.4    162892
2.5    103918
2.1     35518
3.2     15864
3.1      8486
3.3      2713
Name: labels, dtype: int64

In [46]:
labels = ['1.2','1.3','2.2','1.1','2.3','1.4','2.4','2.5','2.1','3.2','3.1','3.3']

In [42]:
a.values[0]

In [88]:
data[(data['labels'].isna())][data['SumSumma']==0]['PartnerID'].to_csv('ZeroesCL.csv')

In [82]:
data[(data['labels'].isna())]

,PartnerID,Age,SEX,CityID,AvgDneyPrihodov,LifeTimeM,QntPurchaseCode,SumQuantity,SUMisPartnerPromotionID,PercentisPartnerPromotionID,AVGisPartnerPromotionID,SUMisCrossPromo,PercentisCrossPromo,AVGisCrossPromo,AVGPercentinSMS,PercentMonth1,PercentMonth2,PercentMonth3,PercentMonth4,PercentMonth5,PercentMonth6,PercentMonth7,PercentMonth8,PercentMonth9,PercentMonth10,PercentMonth11,PercentMonth12,PercentNY,PercentDR,PercentVD,Percent8M,SUMPrice,AVGPrice,SumSumma,AVGSumma,SumSummaDiscount,AvgSummaDiscount,AVGProcentSkidki,PercentMen,SumChild,PercentChild,PercentKolco,PercentSERGI,PercentPODVES,PercentFUTLAR,PercentCEP,PercentBRASLET,PercentKOLE,PercentNKNOWN_WEAR,AVGCostPrice,AVGCostMin,AVGCostTag,AVGCostStock,AVGCostRetail,AVGGiftCardPositionID,SumGiftCardPositionID,AVGQntName,PercentExpensive,PercentCheap,PercentPremium,PercentMidle,PercentGold,PercentSilver,PercentNoMetal,labels
229,309,20,1,79,0,70,1,1,0,0,0,0,0.0,0,,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,971,971.0000,971.0,971.0000,809.0,809.0000,0.5455,0.0,0,0.0,0.0,0.0,1.000000,0.00,0.000000,0.000000,0.0,0.0,413.3100,540.0000,1740.0000,0.0,1159.0000,0,0,1,0.000000,1.000000,0.0,0.00,1.00,0.0,0.0,None
251,342,26,1,29,0,51,1,1,0,0,0,0,0.0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,42339,42339.0000,42339.0,42339.0000,38301.0,38301.0000,0.5250,0.0,0,0.0,0.0,0.0,0.000000,0.00,1.000000,0.000000,0.0,0.0,14989.2000,34172.0000,80640.0000,0.0,42339.0000,0,0,1,1.000000,0.000000,0.0,0.00,1.00,0.0,0.0,None
325,452,33,0,84,0,61,1,1,0,0,0,0,0.0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,2269,2269.0000,2269.0,2269.0000,2081.0,2081.0000,0.5216,0.0,0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.0,811.0100,1531.1900,4350.0000,0.0,2389.0000,0,0,1,0.000000,0.000000,0.0,1.00,1.00,0.0,0.0,None
426,580,39,0,17,0,51,1,1,0,0,0,0,0.0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,16690,16690.0000,16690.0,16690.0000,53310.0,53310.0000,0.2384,0.0,0,0.0,0.0,1.0,0.000000,0.00,0.000000,0.000000,0.0,0.0,11667.9300,16190.0000,70000.0000,16690.0,28000.0000,0,0,1,0.000000,1.000000,0.0,0.00,1.00,0.0,0.0,None
553,787,31,1,105,0,1,0,0,0,0,0,0,0.0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0,0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.0,0.0000,0.0000,0.0000,0.0,0.0000,0,0,0,0.000000,0.000000,0.0,0.00,0.00,0.0,0.0,None
849,1323,39,0,92,0,49,1,1,0,0,0,0,0.0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,10299,10299.0000,10299.0,10299.0000,8981.0,8981.0000,0.5341,0.0,0,0.0,0.0,0.0,0.000000,0.00,0.000000,1.000000,0.0,0.0,4962.2700,8735.0000,19280.0000,0.0,10299.0000,0,0,1,1.000000,0.000000,0.0,0.00,1.00,0.0,0.0,None
941,1490,45,0,92,0,48,1,1,0,0,0,0,0.0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.0,0.0,2719,2719.0000,2719.0,2719.0000,2171.0,2171.0000,0.5560,0.0,0,0.0,1.0,0.0,0.000000,0.00,0.000000,0.000000,0.0,0.0,1027.4000,1740.0000,4890.0000,0.0,2719.0000,0,0,1,0.000000,0.000000,0.0,1.00,1.00,0.0,0.0,None
1052,1681,36,1,91,0,47,1,2,0,0,0,0,0.0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,0.0,6457,6457.0000,6457.0,6457.0000,2682.0,2682.0000,0.7065,0.0,0,0.0,0.0,0.0,0.000000,0.50,0.000000,0.500000,0.0,0.0,2182.2200,3855.0000,9139.0000,199.0,6457.0000,0,0,2,0.000000,0.500000,0.0,0.50,0.50,0.0,0.0,None
1077,1725,36,1,92,0,47,2,3,0,0,0,0,0.0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.500000,0.0,0.0,0.0,1568,784.0000,1568.0,784.0000,962.0,481.0000,0.6137,0.0,0,0.0,0.0,0.0,0.333333,0.00,0.333333,0.333333,0.0,0.0,248.8650,418.5000,1265.0000,0.0,784.0000,0,0,1,0.333333,0.666667,0.0,0.00,0.00,1.0,0.0,None
1541,2482,25,0,92,0,49,1,1,0,0,0,0,0.0,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.0,2249,2249.0000,2249.0,2249.0000,2441.0,2441.0000,0.4795,0.0,0,0.0,0.0,1.0,0.000000,0.00,0.000000,0.000000,0.0,0.0,953.6200,1688.0800,4690.0000,0.0

In [149]:
train = data.dropna()
y_train = train['labels']
train.drop(['labels','AVGPercentinSMS'],inplace=True,axis=1)

In [150]:
y_train.value_counts()

1.2    612507
1.3    473692
2.2    373053
1.1    302595
2.3    254818
1.4    175388
2.4    162892
2.5    103918
2.1     35518
3.2     15864
3.1      8486
3.3      2713
Name: labels, dtype: int64

In [153]:
y_train

0          5
1          4
2          0
3          0
4          1
5          2
6          0
7          4
8          5
9          3
10         2
11         2
12         0
13         3
14         0
15         0
16         2
17         4
18         0
19         4
20         0
21         0
22         0
23         2
24         1
25         4
26         2
27         2
28         2
29         0
          ..
2718212    1
2718213    0
2718215    0
2718216    1
2718217    1
2718218    1
2718219    1
2718221    0
2718222    0
2718223    0
2718224    5
2718225    1
2718226    0
2718227    3
2718228    0
2718229    5
2718230    1
2718232    1
2718234    0
2718235    0
2718236    0
2718237    1
2718238    1
2718239    0
2718240    3
2718241    0
2718242    0
2718243    1
2718244    0
2718245    3
Name: labels, Length: 2521444, dtype: int64

In [152]:
y_train = y_train.map({'1.2':0,'1.3':1,'2.2':2,'1.1':3,'2.3':4,'1.4':5,'2.4':6,'2.5':7,'2.1':8,'3.2':9,'3.1':10,'3.3':11})

In [138]:
test = (data[(data['labels'].isna())])
test = test[test['SumSumma']>0].drop(['labels','AVGPercentinSMS'],axis=1)

In [139]:
print(train.shape)
print(test.shape)

(2521444, 63)
(178836, 63)


In [140]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2521444 entries, 0 to 2718245
Data columns (total 63 columns):
PartnerID                      int64
Age                            int64
SEX                            int64
CityID                         int64
AvgDneyPrihodov                int64
LifeTimeM                      int64
QntPurchaseCode                int64
SumQuantity                    int64
SUMisPartnerPromotionID        int64
PercentisPartnerPromotionID    int64
AVGisPartnerPromotionID        int64
SUMisCrossPromo                int64
PercentisCrossPromo            float64
AVGisCrossPromo                int64
PercentMonth1                  float64
PercentMonth2                  float64
PercentMonth3                  float64
PercentMonth4                  float64
PercentMonth5                  float64
PercentMonth6                  float64
PercentMonth7                  float64
PercentMonth8                  float64
PercentMonth9                  float64
PercentMonth10  

In [141]:
from catboost import CatBoostClassifier
from catboost import Pool

In [165]:
train_pool = Pool(train, y_train)
test_pool = Pool(test)
# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=200, loss_function='MultiClass', random_seed=12345)#, learning_rate=1, depth=2, loss_function='MultiClass') #,calc_feature_importance=True)
# Fit model
model.fit(train_pool)



0:	learn: -2.2545972	total: 13.1s	remaining: 43m 29s
1:	learn: -2.0865873	total: 25s	remaining: 41m 14s
2:	learn: -1.9537518	total: 37.5s	remaining: 41m
3:	learn: -1.8438777	total: 51.1s	remaining: 41m 45s
4:	learn: -1.7504349	total: 1m 3s	remaining: 41m 14s
5:	learn: -1.6692723	total: 1m 18s	remaining: 42m 7s
6:	learn: -1.5975605	total: 1m 31s	remaining: 41m 52s
7:	learn: -1.5338289	total: 1m 43s	remaining: 41m 19s
8:	learn: -1.4721192	total: 1m 54s	remaining: 40m 24s
9:	learn: -1.4197250	total: 2m 5s	remaining: 39m 38s
10:	learn: -1.3681726	total: 2m 18s	remaining: 39m 34s
11:	learn: -1.3214919	total: 2m 29s	remaining: 39m 1s
12:	learn: -1.2788568	total: 2m 41s	remaining: 38m 45s
13:	learn: -1.2403549	total: 2m 54s	remaining: 38m 37s
14:	learn: -1.2032497	total: 3m 5s	remaining: 38m 8s
15:	learn: -1.1689551	total: 3m 17s	remaining: 37m 49s
16:	learn: -1.1366784	total: 3m 27s	remaining: 37m 18s
17:	learn: -1.1066836	total: 3m 38s	remaining: 36m 52s
18:	learn: -1.0782540	total: 3m 52s	

148:	learn: -0.3262546	total: 32m	remaining: 10m 57s
149:	learn: -0.3252337	total: 32m 13s	remaining: 10m 44s
150:	learn: -0.3241106	total: 32m 29s	remaining: 10m 32s
151:	learn: -0.3228150	total: 32m 43s	remaining: 10m 20s
152:	learn: -0.3220303	total: 32m 55s	remaining: 10m 6s
153:	learn: -0.3213625	total: 33m 8s	remaining: 9m 53s
154:	learn: -0.3204644	total: 33m 20s	remaining: 9m 40s
155:	learn: -0.3194314	total: 33m 34s	remaining: 9m 28s
156:	learn: -0.3187409	total: 33m 48s	remaining: 9m 15s
157:	learn: -0.3178521	total: 34m 3s	remaining: 9m 3s
158:	learn: -0.3170274	total: 34m 14s	remaining: 8m 49s
159:	learn: -0.3160834	total: 34m 28s	remaining: 8m 37s
160:	learn: -0.3152017	total: 34m 40s	remaining: 8m 24s
161:	learn: -0.3142049	total: 34m 54s	remaining: 8m 11s
162:	learn: -0.3135350	total: 35m 6s	remaining: 7m 58s
163:	learn: -0.3126006	total: 35m 22s	remaining: 7m 45s
164:	learn: -0.3120012	total: 35m 36s	remaining: 7m 33s
165:	learn: -0.3111418	total: 35m 48s	remaining: 7m 

In [166]:
pred = model.predict(train)

In [167]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train, pred.astype(int))

0.8965549105988473

In [168]:
pred_new = model.predict(test)

In [172]:
pred_new = pred_new.astype(int)

In [186]:
new_client = pd.concat([test.reset_index()['PartnerID'],pd.DataFrame(pred_new)],axis=1,ignore_index=True)
new_client.columns = ('PartnerID','labels')

In [190]:
new_client['labels'] = new_client['labels'].map({0:'1.2',1:'1.3',2:'2.2',3:'1.1',4:'2.3',5:'1.4',6:'2.4',7:'2.5',8:'2.1',9:'3.2',10:'3.1',11:'3.3'})

In [204]:
all_clients = pd.concat([new_client,data.dropna()[['PartnerID','labels']]],axis=0,ignore_index=True)

In [207]:
all_clients.shape

(2700280, 2)

In [208]:
data.shape

(2778321, 65)

In [210]:
all_clients[['PartnerID','labels']].to_csv('RetailClasters2.csv')